In [ ]:
#20180801 Trial 1: 去除训练集中异常数据（p异常），将测试集中异常数据按照lightgbm模型拉回真实值附近

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import lightgbm

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

%matplotlib inline

In [2]:
rep_cols = {'ID':'ID', 
 '板温':'board_t', 
 '现场温度':'env_t', 
 '光照强度':'light_strength', 
 '转换效率':'efficiency', 
 '转换效率A':'efficiency_A', 
 '转换效率B':'efficiency_B', 
 '转换效率C':'efficiency_C', 
 '电压A':'V_A',
 '电压B':'V_B', 
 '电压C':'V_C', 
 '电流A':'I_A', 
 '电流B':'I_B', 
 '电流C':'I_C', 
 '功率A':'P_A', 
 '功率B':'P_B', 
 '功率C':'P_C', 
 '平均功率':'P_avg', 
 '风速':'wind_speed',
 '风向':'wind_direction', 
 '发电量':'y'
}

In [3]:
def my_val(preds, train_data):
    label = train_data.get_label()
    return 'score', 1/(1+np.sqrt(mean_squared_error(preds, label))), True

In [4]:
def my_obj(preds, train_data):
    labels = train_data.get_label()

In [27]:
train_raw = pd.read_csv('../data/public.train.csv')
test_raw = pd.read_csv('../data/test_corrected_lightgbm.csv')

train_raw_len = train_raw.shape[0]

#为了统一处理表头编码问题，合并
train_raw.rename(index=str, columns=rep_cols, inplace=True)

train_test_raw = pd.concat([train_raw, test_raw],sort=False)



train_test_raw.head()

,ID,board_t,env_t,light_strength,efficiency,efficiency_A,efficiency_B,efficiency_C,V_A,V_B,...,I_A,I_B,I_C,P_A,P_B,P_C,P_avg,wind_speed,wind_direction,y
0,10,-19.14,-17.4,34,80.55,106.32,16.98,118.36,729.0,709.0,...,1.34,0.22,1.50,976.86,155.98,1087.50,740.11,0.6,272,1.437752
1,11,-18.73,-17.3,30,99.90,139.00,21.20,139.51,728.0,717.0,...,1.55,0.24,1.56,1128.40,172.08,1132.56,811.01,0.8,275,1.692575
2,12,-17.54,-17.0,41,82.48,114.86,14.91,117.66,731.0,722.0,...,1.75,0.23,1.82,1279.25,166.06,1310.40,918.57,1.1,283,1.975787
3,14,-15.43,-16.6,53,73.98,101.72,15.55,104.67,730.0,727.0,...,2.02,0.31,2.09,1474.60,225.37,1517.34,1072.44,0.9,280,2.370656
4,15,-14.60,-16.3,65,64.62,86.86,13.09,93.92,727.0,729.0,...,2.13,0.32,2.30,1548.51,233.28,1674.40,1152.06,1.1,280,2.532091


In [28]:
#覆盖**_raw，数据内容未变，表头已转为英文
train_raw = train_test_raw.iloc[0:train_raw_len]
test = train_test_raw.iloc[train_raw_len:]

In [29]:
#引入工程知识，去除P_avg大于20000的记录
train = train_raw.drop(train_raw[train_raw.P_avg>20000].index)

print(str(train_raw.shape[0]-train.shape[0])+" samples have been removed due to extremely exceptional average power value")

23 samples have been removed due to extremely exceptional average power value


In [30]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': 'mse',
    'num_leaves': 31,
    'learning_rate': 0.05,
    'feature_fraction': 0.9,
    'bagging_fraction': 0.8,
    'bagging_freq': 5,
    'verbose': 0
}

predictor = [c for c in train.columns.tolist() if c!='y' and c!='ID']

kf = KFold(5, shuffle=True, random_state=2018)

test_predicts = []
val_preds = np.zeros(train.shape[0])
for n_fold, (tra_idx, val_idx) in enumerate(kf.split(train)):
    tra = train.iloc[tra_idx]
    val = train.iloc[val_idx]
    
    train_set = lightgbm.Dataset(
        tra[predictor],
        tra['y']
    )
    
    validation_set = lightgbm.Dataset(
        val[predictor],
        val['y']
    )
    
    model = lightgbm.train(params, train_set, num_boost_round=5000,
                          valid_sets= [validation_set],
                          valid_names=['valid'],
                          early_stopping_rounds=100,
                           feval=my_val,
                          verbose_eval=100)

    val_preds[val_idx] = model.predict(val[predictor])
    test_predicts.append(model.predict(test[predictor]))
print(1/(1+np.sqrt(mean_squared_error(train['y'],val_preds))))

Training until validation scores don't improve for 100 rounds.
[100]	valid's l2: 0.0541882	valid's score: 0.811172
[200]	valid's l2: 0.0513327	valid's score: 0.815284
[300]	valid's l2: 0.0509408	valid's score: 0.81586
[400]	valid's l2: 0.050613	valid's score: 0.816344
[500]	valid's l2: 0.0505113	valid's score: 0.816495
[600]	valid's l2: 0.0504223	valid's score: 0.816627
[700]	valid's l2: 0.0503838	valid's score: 0.816684
[800]	valid's l2: 0.0502833	valid's score: 0.816834
[900]	valid's l2: 0.0502536	valid's score: 0.816878
[1000]	valid's l2: 0.0502591	valid's score: 0.81687
Early stopping, best iteration is:
[919]	valid's l2: 0.0502482	valid's score: 0.816886
Training until validation scores don't improve for 100 rounds.
[100]	valid's l2: 0.0332444	valid's score: 0.845787
[200]	valid's l2: 0.0302955	valid's score: 0.851748
[300]	valid's l2: 0.0295056	valid's score: 0.853408
[400]	valid's l2: 0.0289511	valid's score: 0.854591
[500]	valid's l2: 0.0286674	valid's score: 0.855202
[600]	val

In [31]:
test['ans'] = np.mean(test_predicts, axis=0)
test[['ID','ans']].to_csv('../result/080101.csv',header=False, index=False)

C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [34]:
a = pd.read_csv('../result/073102_rm_excep_correct_test_lm.csv')
b = pd.read_csv('../result/080101.csv')


In [39]:
b.head()

,1,0
0,9,1.269764
1,13,2.154524
2,17,3.395164
3,18,3.665256
4,21,4.146404


In [40]:
diff = a.iloc[:,0:] - b.iloc[:,0:]
diff.head(50)

,1,0
0,0,-0.012818
1,0,0.009302
2,0,0.010183
3,0,-0.014961
4,0,0.021913
5,0,0.091001
6,0,-0.031448
7,0,-0.032441
8,0,0.050865
9,0,0.037763


In [41]:
diff.describe()

,1,0
count,8408.0,8408.000000
mean,0.0,-0.006576
std,0.0,0.175207
min,0.0,-5.014740
25%,0.0,-0.016199
50%,0.0,-0.000433
75%,0.0,0.016237
max,0.0,3.052294
